# Creating a Turboprop Powered Aircraft Example

Here we will create an aircraft using a more advanced `EngineModel`, the `TurbopropModel`. This is a more complicated type of engine that combines several different models into a single unit. The `TurbopropModel` has three individual pieces needed to create it - a "shaft power model" that describes how the turboprop turns energy into shaft power, such as a turboshaft or motor, a "gearbox model" for adjusting the shaft RPM, and a "propeller model" to convert shaft power into thrust. Let's walk through two examples: first building a turboprop-powered aircraft with a minimum amount of information, and then a second example where we electrify the aircraft by replacing the default models in the `TurbopropModel` with custom ones.

## Basic Turboprop Aircraft Example

In [2]:
import aviary.api as av
from aviary.models.aircraft.large_turboprop_freighter.phase_info import two_dof_phase_info

Aircraft = av.Aircraft
Mission = av.Mission
Dynamic = av.Dynamic

# define the minimum option set for a turboprop
options = av.AviaryValues()

# top-level turboprop settings
options.set_val(av.Settings.VERBOSITY, 0)  # quiet unneeded printouts
options.set_val(Aircraft.Engine.FIXED_RPM, 13820, units='rpm')

# EngineDeck minimum option set
options.set_val(Aircraft.Engine.DATA_FILE, av.get_path('models/engines/turboshaft_4465hp.csv'))

# Gearbox model minimum option set
options.set_val(Aircraft.Engine.Gearbox.GEAR_RATIO, 13.55, 'unitless')
options.set_val(Aircraft.Engine.Gearbox.SHAFT_POWER_DESIGN, 4465, 'hp')

# Hamilton Standard propeller minimum option set
options.set_val(Aircraft.Engine.Propeller.TIP_MACH_MAX, 1.0)
options.set_val(Aircraft.Engine.Propeller.NUM_BLADES, val=4, units='unitless')
options.set_val(Aircraft.Engine.Propeller.COMPUTE_INSTALLATION_LOSS, True)

# Initialize turboprop model. Model uses an EngineDeck built from `options`, basic
# gearbox model with default efficiency of 1, and the Hamilton Standard propeller model
# "turboprop" is ready to be included in an AviaryProblem
turboprop = av.TurbopropModel(name='turboprop_example', options=options)

# Build and run AviaryProblem using the Level2 interface
prob = av.AviaryProblem()

prob.load_inputs(
    'models/aircraft/large_turboprop_freighter/large_turboprop_freighter_GASP.csv',
    two_dof_phase_info,
)

prob.load_external_subsystems([turboprop])

prob.check_and_preprocess_inputs()

prob.build_model()
prob.add_driver('IPOPT', max_iter=0, verbosity=0)
prob.add_design_variables()
prob.add_objective()
prob.setup()
prob.run_aviary_problem(suppress_solver_print=True, make_plots=False)


'rhs_checking' is disabled for 'DirectSolver in 'traj.phases.ascent' <class Phase>' but that solver has redundant adjoint solves. If it is expensive to compute derivatives for this solver, turning on 'rhs_checking' may improve performance.


'rhs_checking' is disabled for 'DirectSolver in 'traj.phases.climb1' <class Phase>' but that solver has redundant adjoint solves. If it is expensive to compute derivatives for this solver, turning on 'rhs_checking' may improve performance.


'rhs_checking' is disabled for 'DirectSolver in 'traj.phases.climb2' <class Phase>' but that solver has redundant adjoint solves. If it is expensive to compute derivatives for this solver, turning on 'rhs_checking' may improve performance.


'rhs_checking' is disabled for 'DirectSolver in 'traj.phases.desc1' <class Phase>' but that solver has redundant adjoint solves. If it is expensive to compute derivatives for this solver, turning on 'rhs_checking' may improve performance.



/home0/jkirk5/miniforge3/envs/aviary/lib/python3.12/site-packages/openmdao/core/total_jac.py:1698: DerivativesWarning:The following design variables have no impact on the constraints or objective at the current design point:
  aircraft:engine:gearbox:shaft_power_design, inds=[0]
  aircraft:engine:propeller:integrated_lift_coefficient, inds=[0]



Aviary run failed. See the dashboard for more details.




## Electrified Propeller Aircraft Example

This model is a WIP and will be coming soon